<a href="https://colab.research.google.com/github/Vilmo18/Languages_Models_Without_Entity_Knowledge/blob/main/Pretrain_gpt2_without_entity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets flair -q

# Anonymization

In [11]:
from flair.data import Sentence
from flair.models import SequenceTagger
from datasets import load_dataset
import random
import re

tagger = SequenceTagger.load('flair/ner-english-ontonotes-large')
dataset = load_dataset("abisee/cnn_dailymail", "3.0.0")
included_tags = {"EVENT", "FAC", "GPE", "LANGUAGE", "LAW", "LOCATION", "NORP",
                 "ORG", "PERSON", "PRODUCT", "WORK_OF_ART"}


pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [12]:
import re

def clean_text(text):
    # Define a regex pattern to match emojis and various symbols
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F700-\U0001F77F"  # alchemical symbols
        u"\U0001F780-\U0001F7FF"  # other symbols & pictographs
        u"\U0001F800-\U0001F8FF"  # additional symbols & pictographs (2)
        u"\U0001F900-\U0001F9FF"  # supplemental symbols & pictographs
        u"\U0001FA00-\U0001FA6F"  # other supplemental symbols
        u"\U0001FA70-\U0001FAFF"  # other supplemental symbols (2)
        u"\U00002702-\U000027B0"  # miscellaneous symbols
        u"\U000024C2-\U0001F251"  # enclosed characters
        "]+", flags=re.UNICODE
    )

    # Remove emojis and pictographic symbols
    text = emoji_pattern.sub(r'', text)

    # Remove unidentified characters (e.g., non-ASCII characters)
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    return text

In [23]:
def clean_dataset(dataset):
    return dataset.map(lambda x: {'article': clean_text(x['article']),
                                  'highlights': clean_text(x['highlights'])},num_proc=8)

new_ds = DatasetDict({
    'train': clean_dataset(dataset['train']),
    'validation': clean_dataset(dataset['validation']),
    'test': clean_dataset(dataset['test'])
})


In [27]:
def anonymize_text(text, tagger):
    sentence = Sentence(text)
    tagger.predict(sentence)

    entities = sentence.get_spans('ner')

    replacements = []
    for entity in entities:
        if entity.tag in included_tags:
            start_pos = entity.start_position
            end_pos = entity.end_position
            anon_id = f"ent{random.randint(1, 1800000)}"
            replacements.append((start_pos, end_pos, anon_id))

    replacements.sort(reverse=True, key=lambda x: x[0])

    for start, end, anon_id in replacements:
        text = text[:start] + anon_id + text[end:]

    return text

def anonymize_example(example):
    example['article'] = anonymize_text(example['article'], tagger)
    example['highlights'] = anonymize_text(example['highlights'], tagger)
    return example


In [28]:
# Testing
train_subset = new_ds['train'].select(range(5))
anonymized_subset = train_subset.map(anonymize_example)

for i in range(5):
    print(anonymized_subset[i])

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

{'article': 'ent1122893, ent265684 (ent1341752) -- ent1049114 star ent379177 gains access to a reported 20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. ent1609244 as ent1155978 in ent471992 To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an ent192619 interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, ent1444126 will be able to gamble in a casino, buy a drink in a pub or see the horror film "ent1049113," currently six places below his number one movie on the ent688815 box office chart. Details of how he\'ll mark his landmar

In [ ]:
anonymized_dataset = new_ds.map(anonymize_example)

In [ ]:
# anonymized_dataset.push_to_hub('cnn-news-anonymised')


# Tokenizer

In [2]:
from datasets import load_dataset
repo = load_dataset("YvanCarre/cnn-news-anonymised")
repo

README.md:   0%|          | 0.00/622 [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/256M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/256M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [3]:
from tqdm import tqdm

with open("all_articles.txt", "w") as corpus:
    for dataset_name in repo:
        for text in tqdm(repo[dataset_name], desc=f"Processing {dataset_name}"):
            corpus.write(text["article"] + "\n")

print("completed")

Processing test: 100%|██████████| 11490/11490 [00:00<00:00, 16406.14it/s]

completed


In [ ]:
from tokenizers import SentencePieceUnigramTokenizer
from tokenizers import Tokenizer
tokenizer = SentencePieceUnigramTokenizer()
tokenizer.train(files="all_articles.txt", vocab_size=32000)

In [4]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, processors,decoders
from tokenizers.models import Unigram
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import UnigramTrainer

tokenizer = Tokenizer(Unigram())

special_tokens = ["<|endoftext|>", "<unk>", "<pad>"]

tokenizer.pre_tokenizer = ByteLevel()
trainer = UnigramTrainer(
    vocab_size=32000,
    special_tokens=special_tokens,
    unk_token="<unk>"
)

files = ["all_articles.txt"]
tokenizer.train(files, trainer)

tokenizer.post_processor = processors.ByteLevel(trim_offsets=True)
tokenizer.decoder = decoders.ByteLevel()

tokenizer.save("unigram_tokenizer.json")


In [5]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

In [6]:
tokenizer

GPT2TokenizerFast(name_or_path='', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [ ]:
# tokenizer.push_to_hub('anonym-gpt2')

# Pretraining

In [ ]:
from transformers import AutoConfig, AutoModelForCausalLM
from transformers import AutoTokenizer

config = AutoConfig.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_config(config)
new_vocab_size = 32000
model.resize_token_embeddings(new_vocab_size)

## Custum tokenizer
tokenizer = AutoTokenizer.from_pretrained("YvanCarre/anonym-gpt2")

tokenizer_config.json:   0%|          | 0.00/834 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

repo = load_dataset("YvanCarre/cnn-news-anonymised")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['article'], truncation=True, max_length=512)

tokenized_dataset = repo.map(tokenize_function, batched=True, remove_columns=['article','highlights', 'id'],num_proc=8)
tokenized_dataset

Map (num_proc=8):   0%|          | 0/287113 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/13368 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 11490
    })
})

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from transformers import TrainingArguments,Trainer

training_args = TrainingArguments(
    output_dir="./gpt2",
    overwrite_output_dir=True,
    max_steps=30000,
    per_device_train_batch_size=10,
    evaluation_strategy="steps",
    eval_steps=4000,
    save_steps=4000,
    #save_total_limit=3,
    logging_dir='./logs',
    load_best_model_at_end=True,
    logging_steps=2000,
    learning_rate=1e-3,
    weight_decay=0.01,
    adam_beta1=0.9,
    adam_beta2=0.999,
    gradient_accumulation_steps=2,
    report_to='none',
    fp16=True,
    #max_grad_norm=1.0,
    lr_scheduler_type="cosine",
    warmup_steps=8000,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: carre (carre-su). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


In [ ]:
max_length = 100
temperature = 0.7
top_k = 10
top_p = 0.95
repetition_penalty = 1.2
num_return_sequences = 5

In [ ]:
input_text = "This is my life"

input_ids = tokenizer.encode(input_text, return_tensors="pt")
input_ids = input_ids.to(model.device)

output_pretrained = model.generate(
    input_ids,
    max_length=max_length,
    temperature=temperature,
    top_k=top_k,
    top_p=top_p,
    repetition_penalty=repetition_penalty,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True,
    num_return_sequences=num_return_sequences)


for i, output in enumerate(output_pretrained):
    output_text_pretrained = tokenizer.decode(output, skip_special_tokens=True)
    print(f"Generated Text {i+1}:\n", output_text_pretrained, "\n")


Generated Text 1:
 This is my life-changing ent403914 video that has captured the moment I finally feel like a woman. The video, entitled 'ent623169', starts with a montage of women in bikinis and on to their underwear at a time when they are all ready for what they are doing. As soon as it hits, I begin feeling like a woman and I start shaking uncontrollably. Scroll down for 

Generated Text 2:
 This is my life-changing moment at the age of 85. For me, the time has come to get used to being in ent630's most prestigious school for girls and boys. The school was founded by Sir ent777672 on behalf of ent1412862 after his wife died from cancer two years ago. It was a place where all of us were taught about ent1578045 history and culture. Now, nearly three decades later, I 

Generated Text 3:
 This is my life-size dream of a baby elephant. It was born in ent1064860, and I have always dreamed of living there for the first time. But then on Saturday morning it was revealed that the world’s l

Designed as a walkthrough with this [YouTube video](https://colab.research.google.com/drive/1y0KnCFZvGVf_odSfcNAws6kcDD7HsI0L?usp=sharing)

In [ ]:
# model.push_to_hub("YvanCarre/anonym-gpt2")

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/YvanCarre/gpt2_anonym/commit/10110495ccae5b693bbd25690ac327490c2e562c', commit_message='Upload model', commit_description='', oid='10110495ccae5b693bbd25690ac327490c2e562c', pr_url=None, pr_revision=None, pr_num=None)